# Understanding the basic concepts

In [ ]:
import siibra
from nilearn import plotting

## Spaces, parcellations and regions

At its core, `siibra` defines different maps and reference spaces as objects. These can be accessed in the form of predefined entities, which autocomplete in most python editors, or via fuzzy string matching. Let's try a few.

In [ ]:
print(siibra.spaces)

In [ ]:
print(siibra.parcellations)

In [ ]:
space = siibra.spaces.BIG_BRAIN
space = siibra.spaces['bigbrain'] # this is equivalent
print(type(space),space.name)

These objects are typically linked to curated datasets in the [EBRAINS knowledge graph](https://kg.ebrainseu) (KG), using dataset identifiers.

In [ ]:
parcellation = siibra.parcellations.JULICH_BRAIN_CYTOARCHITECTONIC_MAPS_2_9
print(parcellation.id)
print(parcellation.name)
print(parcellation.description)

`siibra` also represents brain regions as semantic objects using the specific class `Region`. Since brain regions are usually part of a hierarchy, each region represents a tree which can have parent and child nodes. We can search regions from a parcellation using its `find` method. If we are confident that a region exists, we can request a unique object using `decode`, which will raise an Exception if the specification is not unique.  

In [ ]:
region = parcellation.decode_region('v1')

print("Region found:",region)
print("Its parent:",region.parent)

## Maps

Spaces and parcellations are just semantic objects - they define a concept and provide desriptions. We are typically interested to combine them, i.e. to work with a concrete parcellation map in a particular reference space. Since EBRAINS atlases provide maps of some parcellations in multiple spaces, maps and parcellations are different concepts - maps are spatial objects, while parcellations are pure semantic objects.

In [ ]:
cytomap = parcellation.get_map(siibra.spaces['mni152'])
print(cytomap)
print(f"This object defines {len(cytomap)} maps.")

`siibra` typically performs lazy loading of data, so that data is only retrieved when requested. As a consequence, in order to access the actual parcellation image, we call `fetch`, or `fetchall` if the parcellation defines multiple maps. The resulting objects are a `nibabel` standard type, and can thus be directly used in many common software libraries. Here we display the maximum probability map of the Julich-Brain probabilistic maps, which comes in separate images for the left and right hemisphere.

In [ ]:
from nilearn import plotting
for img in  cytomap.fetchall():
    plotting.plot_stat_map(img)

In [ ]:
# Try to display maps of long fiber bundles in mni152 space!
bundlemap = ..
plotting.plot_stat_map(bundlemap)

Probabilistic atlases provide a richer continuous representation of each areal map, whicih can be accessed by explicitly selecting the continuous map type `siibra.MapType.CONTINUOUS` when requesting the map.

In [ ]:
pmaps = siibra.parcellations['julich 2.9'].get_map(siibra.spaces['mni152'],siibra.MapType.CONTINUOUS)
print(pmaps)
print(f"This map defines {len(pmaps)} different maps.")

Of course, maps and label indices of maps do represent brain regions. `siibra`'s map objects keep track of label and map indices appropriately, so we can always access the corresponding region object. This is crucial to avoid confusion between indices and regions, a common error source in data experiments that employ brain atlases.

As an example, assume we want to display the probabiliy map of v1 in the left hemisphere. Which one to fetch?

In [ ]:
# Which probability map encodes V1 in the left hemisphere?
index = pmaps.decode_region('v1 left')
plotting.plot_stat_map(pmaps.fetch(mapindex=index[0].map))

Vice versa, we can easily find the region of a given map index.

In [ ]:
region = pmaps.decode_label(120)
print(region)

The same works accordingly for the maximum probabily map, where the region is encoded by both the map index (here: the hemisphere) and its color, i.e. label index: 

In [ ]:
cytomap.decode_label(mapindex=0,labelindex=10)

## Reference templates and high-resolution data

Besides brain maps, each reference space also has a data representation - typically an image - which represents the space. Interesting is particularly the BigBrain, which is a microscopic model and too large to download. For such cases, `siibra` allows to fetch high-resolution data at reduced resolutions, or fetching only volumes of interest.

In [ ]:
mni152 = siibra.spaces['mni152'].get_template()
plotting.plot_img(mni152.fetch(),cmap='gray')

In [ ]:
bigbrain = siibra.spaces['bigbrain'].get_template()
plotting.view_img(bigbrain.fetch(),bg_img=None,cmap='gray')

In [ ]:
minpt = (-3.979, -61.256, 3.906)
maxpt = (5.863, -55.356, -2.487)
bb = siibra.spaces['bigbrain']
voi = siibra.spaces['bigbrain'].get_voi(minpt,maxpt)
chunk = bigbrain.fetch(resolution_mm=-1, voi=voi) # -1 means: highest resolution available
plotting.view_img(chunk,bg_img=None,cmap='gray')

We can reuse this region of interest, to fetch maps of cortical layers and display them with the BigBrain data.

In [ ]:
layers = siibra.parcellations['layers'].get_map("bigbrain").fetch(resolution_mm=-1,voi=voi)
plotting.view_img(layers,bg_img=chunk,opacity=.1,symmetric_cmap=False)

# Atlas objects

Dealing with spaces, parcellations and regions is much simplified by using an atlas object. Atlases provide  relationships between related spaces and parcellations, and make it easier to work with them.

In [ ]:
atlas = siibra.atlases.MULTILEVEL_HUMAN_ATLAS
atlas.select_region('v1')
print(atlas.selected_region)
m = atlas.build_mask(siibra.spaces.MNI152_2009C_NONL_ASYM)
plotting.plot_roi(m)